In [1]:
import eeg_decoder 
import pandas as pd
import numpy as np
import scipy.io as io
from pathlib import Path
import shutil

In [2]:
experiment = 'e'
data_dir = f'./data/{experiment}'
exp = eeg_decoder.Experiment(experiment, data_dir, dev=False,info_from_file=False)
sub_folders = [f for f in Path(exp.data_dir).iterdir() if f.is_dir()]
sub_folders

[WindowsPath('data/e/01'),
 WindowsPath('data/e/02'),
 WindowsPath('data/e/03'),
 WindowsPath('data/e/04'),
 WindowsPath('data/e/05'),
 WindowsPath('data/e/06'),
 WindowsPath('data/e/07'),
 WindowsPath('data/e/08'),
 WindowsPath('data/e/10')]

In [3]:
sub_folders = sub_folders[-2:]

In [4]:
sub_folders

[WindowsPath('data/e/08'), WindowsPath('data/e/10')]

In [6]:
for isub, sub_folder in enumerate(sub_folders):

    exp.xdata_files = list(sub_folder.rglob('*xdata.mat'))
    exp.ydata_files = list(sub_folder.rglob('*ydata.mat'))
    exp.behavior_files = list(sub_folder.rglob('*behavior.csv'))
    exp.artifact_idx_files = list(sub_folder.rglob('*artifact_idx.mat'))        

    # load and concatenate data from behavior, eeg, and trial labels
    for irun in range(len(exp.xdata_files)): 
        x1, y1 = exp.load_eeg(irun)
        idx1 = exp.load_artifact_idx(irun)
        beh1 = exp.load_behavior(irun, remove_artifact_trials=False)
        print(len(pd.DataFrame(beh1)), len(x1), len(y1), len(idx1), sum(idx1))

        if irun == 0:
            x = x1 
            y = y1
            idx = idx1
            beh = pd.DataFrame(beh1)
            sub_num = sub_folder.name
        else:
            x = np.concatenate((x, x1), 0)
            y = np.concatenate((y, y1), 0)
            idx = np.concatenate((idx, idx1), 0)
            beh1_df = pd.DataFrame(beh1)
            beh = pd.concat((beh, beh1_df))

    x = np.moveaxis(x, 0, 2) # roll bc saving with savemat changes shape for some reason...

    # save
    io.savemat(f'data/{experiment}/{experiment}_{sub_num}_xdata.mat',{'xdata':x})
    io.savemat(f'data/{experiment}/{experiment}_{sub_num}_ydata.mat',{'ydata':y})
    io.savemat(f'data/{experiment}/{experiment}_{sub_num}_artifact_idx.mat',{'artifact_idx':idx})
    beh.to_csv(f'data/{experiment}/{experiment}_{sub_num}_behavior.csv')
    shutil.copy(f'data/{experiment}/{sub_num}/{experiment}_{sub_num}_1_info.mat',f'data/{experiment}/{experiment}_{sub_num}_info.mat')

229 208 208 229 208
220 212 212 220 212
229 218 218 229 218
231 212 212 231 212
228 218 218 228 218
226 210 210 226 210
226 215 215 226 215
230 215 215 230 215
225 213 213 225 213
226 215 215 226 215
